# Instalação das dependências

In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 17.3MB 213kB/s eta 0:00:01     |███████████████████████████▊    | 15.0MB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 13.0MB 4.2MB/s eta 0:00:01
     |████████████████████████████████| 317kB 31.4MB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
  Found existing installation: numpy 1.16.4
    Uninstalling numpy-1.16.4:
      Successfully uninstalled numpy-1.16.4
  Found existing installation: matplotlib 3.1.1
    Uninstalling matplotlib-3.1.1:
      Successfully uninstalled matplotlib-3.1.1


# pré-processamento dos dados

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
dataset=pd.read_csv('BTC-USD.csv')

training_set=dataset.iloc[:,1:2]        #selecting the second column
training_set.head()                          #print first five rows

,Open
0,375.510010
1,378.248993
2,379.250000
3,381.721985
4,375.717987


In [9]:
training_set=training_set.values            #converting to 2d array
training_set                                 #print the whole data

array([[ 375.51001 ],
       [ 378.248993],
       [ 379.25    ],
       ...,
       [7536.820313],
       [7466.727051],
       [7761.883789]])

In [10]:
#Scaling the data

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()                           #scaling using normalisation 
training_set = sc.fit_transform(training_set)
xtrain=training_set[0:1825]                  #input values of rows [0-1825]		   
ytrain=training_set[1:1826]                  #input values of rows [1-1826]

today_prices=pd.DataFrame(xtrain)               #assigning the values of xtrain to today
tomorrow_prices=pd.DataFrame(ytrain)            #assigning the values of xtrain to tomorrow
ex= pd.concat([today_prices,tomorrow_prices],axis=1)        #concat two columns 
ex.columns=(['today_prices','tomorrow_prices'])
xtrain = np.reshape(xtrain, (1825, 1, 1))     #Reshaping into required shape for Keras